In [1]:
import pandas as pd 
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity


In [5]:
ratings_toy = pd.read_csv('dataset/toy_dataset.csv', index_col=0)
ratings_toy = ratings_toy.fillna(0)
ratings_toy

,action1,action2,action3,romantic1,romantic2,romantic3
user 1,4.0,5.0,3.0,0.0,2.0,1.0
user 2,5.0,3.0,3.0,2.0,2.0,0.0
user 3,1.0,0.0,0.0,4.0,5.0,4.0
user 4,0.0,2.0,1.0,4.0,0.0,3.0
user 5,1.0,0.0,2.0,3.0,3.0,4.0


In [8]:
def standardize(row):
    new_row = (row-row.mean())/(row.max() - row.min())
    return new_row
ratings_toy_std = ratings_toy.apply(standardize)
#ratings_toy_std
# We Transpose the matrix because we want similarity between movies that are needed
#to be in rows
movie_similarity = cosine_similarity(ratings_toy_std.T)
print(movie_similarity)
movie_similarity_df = pd.DataFrame(movie_similarity, index=ratings_toy.columns, columns=ratings_toy.columns)
movie_similarity_df

[[ 1.          0.70668875  0.81368151 -0.79941088 -0.02539184 -0.91410609]
 [ 0.70668875  1.          0.72310153 -0.84515425 -0.5189993  -0.84337386]
 [ 0.81368151  0.72310153  1.         -0.84794611 -0.3799803  -0.80218063]
 [-0.79941088 -0.84515425 -0.84794611  1.          0.14803913  0.72374686]
 [-0.02539184 -0.5189993  -0.3799803   0.14803913  1.          0.39393939]
 [-0.91410609 -0.84337386 -0.80218063  0.72374686  0.39393939  1.        ]]


,action1,action2,action3,romantic1,romantic2,romantic3
action1,1.000000,0.706689,0.813682,-0.799411,-0.025392,-0.914106
action2,0.706689,1.000000,0.723102,-0.845154,-0.518999,-0.843374
action3,0.813682,0.723102,1.000000,-0.847946,-0.379980,-0.802181
romantic1,-0.799411,-0.845154,-0.847946,1.000000,0.148039,0.723747
romantic2,-0.025392,-0.518999,-0.379980,0.148039,1.000000,0.393939
romantic3,-0.914106,-0.843374,-0.802181,0.723747,0.393939,1.000000


In [10]:
#Recommendation Time!!

def get_similar_movies(movie_name, user_rating):
    #subtract 2.5(mean of ratings) from user_rating 
    # If user rating's below 3, movies similar to this movie is pushed down the 
    #preference list
    similar_score = movie_similarity_df[movie_name]*(user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score
print(get_similar_movies('action3',4))

action3      1.500000
action1      1.220522
action2      1.084652
romantic2   -0.569970
romantic3   -1.203271
romantic1   -1.271919
Name: action3, dtype: float64


In [12]:
action_lover = [('action1', 5), ('romantic2', 1), ('romantic3', 1)]
similar_movies = pd.DataFrame()
for movie,rating in action_lover:
    similar_movies = similar_movies.append(get_similar_movies(movie,rating), ignore_index=True)

similar_movies

,action1,action2,action3,romantic1,romantic2,romantic3
0,2.500000,1.766722,2.034204,-1.998527,-0.063480,-2.285265
1,0.038088,0.778499,0.569970,-0.222059,-1.500000,-0.590909
2,1.371159,1.265061,1.203271,-1.085620,-0.590909,-1.500000


In [13]:
similar_movies.sum().sort_values(ascending=False)

action1      3.909247
action2      3.810282
action3      3.807445
romantic2   -2.154389
romantic1   -3.306206
romantic3   -4.376174
dtype: float64